Analog Brickのサンプルコード <br>

101 LED

はじめの定義。

`# coding: utf-8`<br>

を入れることで、コメントに日本語をいれる事が可能になる。

In [1]:
# coding: utf-8

`import`でライブラリを読み込む。<br>
GPIOは“General Purpose Input/Output”の略で、GPIOの操作等のライブラリ。

In [2]:
import RPi.GPIO as GPIO
import time
import sys

LEDを装着するPIN名を記載。GPIO4に装着。

In [3]:
LEDPIN = 4

GPIOをsetmode(GPIO.BCM)で、GPIO.BCMに設定する。<br>
この設定をすることで、GPIO4, GPIO5などの設定でPINを呼び出せる。<br>BroadcomのチップそのもののIDで指定したい場合は、GPIO.setmode(GPIO.BOARD)を呼び出す。

In [4]:
GPIO.setmode(GPIO.BCM)

GPIOの割り振りと対応づけは、 gpio readall のコマンドを呼び出し、確認する。

In [5]:
! gpio readall

 +-----+-----+---------+------+---+---Pi 3---+---+------+---------+-----+-----+
 | BCM | wPi |   Name  | Mode | V | Physical | V | Mode | Name    | wPi | BCM |
 +-----+-----+---------+------+---+----++----+---+------+---------+-----+-----+
 |     |     |    3.3v |      |   |  1 || 2  |   |      | 5v      |     |     |
 |   2 |   8 |   SDA.1 | ALT0 | 1 |  3 || 4  |   |      | 5V      |     |     |
 |   3 |   9 |   SCL.1 | ALT0 | 1 |  5 || 6  |   |      | 0v      |     |     |
 |   4 |   7 | GPIO. 7 |   IN | 1 |  7 || 8  | 0 | IN   | TxD     | 15  | 14  |
 |     |     |      0v |      |   |  9 || 10 | 1 | IN   | RxD     | 16  | 15  |
 |  17 |   0 | GPIO. 0 |   IN | 0 | 11 || 12 | 0 | IN   | GPIO. 1 | 1   | 18  |
 |  27 |   2 | GPIO. 2 |   IN | 0 | 13 || 14 |   |      | 0v      |     |     |
 |  22 |   3 | GPIO. 3 |   IN | 0 | 15 || 16 | 0 | IN   | GPIO. 4 | 4   | 23  |
 |     |     |    3.3v |      |   | 17 || 18 | 0 | IN   | GPIO. 5 | 5   | 24  |
 |  10 |  12 |    MOSI | ALT0 | 0 | 19 |

GPIOのLEDPINをOUTPUTに設定する。

In [45]:
GPIO.setup( LEDPIN, GPIO.OUT )

LEDの点灯/消灯を繰り返す。

while True: で無限ループで繰り返されるので、終了する際は、メニューの■で停止する。<br>
ループ中は、In [*]: になり、停止すると、In [数字]: になる。

In [8]:
 while True:
        GPIO.output( LEDPIN, True )
        time.sleep( 1.0 )
        GPIO.output( LEDPIN, False )
        time.sleep( 1.0 ) 

KeyboardInterrupt: 

---------
102 のサンプル

102 は、RaspberryPIの制御電圧が、3.3Vのために電圧不足で音がならないのでスキップ。

--------
103のサンプル

ボタン用のPINを5に指定する。

In [9]:
BUTTONPIN = 5

ボタン用のPINをGPIO.setup(BUTTONPIN, GPIO.IN) で、GPIO.INに指定する。

In [10]:
GPIO.setup(BUTTONPIN, GPIO.IN)

ボタンが押されたら点灯し、ボタンが消されたら消灯するサンプルを作成する。

In [11]:
while True:
        # ボタン押下判定
        if( GPIO.input(BUTTONPIN)):
            # LED点灯
            GPIO.output(LEDPIN, True)
        else:
            # LED消灯
            GPIO.output(LEDPIN, False)

KeyboardInterrupt: 

-------
104のサンプル

標準のRaspberryPIでは、ADCに対応していない。FaBoのシールドでは、SPI用のチップを搭載する事で、SPIを経由して、ADCを実現している。<br>
ADCは、Analog Digital Converterで、センサーからの電圧値を読み込めるようになる。

最初にspiのライブラリを読み込む。

In [12]:
import spidev

次ぐに、Angleブリックを差し込むGPIOピン名を0に定義する。これは、FaBoオリジナルのPIN番号である。

In [13]:
ANGLEPIN = 0

次に0ピン-5ピンに装着されたBrickのAnalog値を読み込むためのメソッドを作成する。

In [14]:
def readadc(channel):
    adc = spi.xfer2([1,(8+channel)<<4,0])
    data = ((adc[1]&3) << 8) + adc[2]
    return data

次に値の範囲を変更するmap関数のメソッドを作成する。

In [15]:
def map(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min

LEDを装着している4ピンでPWMを扱えるようにする。PWMの周期は、100Hz。

In [16]:
# PWM/100Hzに設定
LED = GPIO.PWM(LEDPIN, 100)

# 初期化
LED.start(0)

SPIの初期化をおこなう

In [17]:
spi = spidev.SpiDev()
spi.open(0,0)

Angleの値が変わったら、LEDの色が変わるサンプルを実行する。

In [18]:
while True:
        data = readadc(ANGLEPIN)
        value = map(data, 0, 1023, 0, 100)
        LED.ChangeDutyCycle(value)
        time.sleep( 0.01 )

KeyboardInterrupt: 

------
105のサンプルは、103と同様に、RaspberryPIの制御電圧が3.3Vのために電圧不足が発生。

-----
106のサンプル

103と同じサンプルを用意する。

In [19]:
while True:
        # ボタン押下判定
        if(GPIO.input(BUTTONPIN)):
            # LED点灯
            GPIO.output(LEDPIN, True)
        else:
            # LED消灯
            GPIO.output(LEDPIN, False)

KeyboardInterrupt: 

----
107のサンプル

103と同じサンプルを用意する。

In [46]:
while True:
        # ボタン押下判定
        if(GPIO.input(BUTTONPIN)):
            # LED点灯
            GPIO.output(LEDPIN, True)
        else:
            # LED消灯
            GPIO.output(LEDPIN, False)

UnboundLocalError: local variable 'led_state' referenced before assignment

toggle


UnboundLocalError: local variable 'led_state' referenced before assignment

KeyboardInterrupt: 

toggle


----
108

108のサンプル

In [47]:
TEMPPIN = 0

In [48]:
while True:
        data = readadc(TEMPPIN)
        volt = map(data, 0, 1023, 0, 5000)
        temp = map(volt, 300, 1600, -30, 100)
        sys.stdout.write("\rtemp = %f" % (temp))
        sys.stdout.flush()
        time.sleep( 0.5 )

temp = 29.000000

KeyboardInterrupt: 

----
109

109のサンプル

In [49]:
LIGHTPIN = 0

In [52]:
while True:
    data = readadc(LIGHTPIN)
    sys.stdout.write("\rambient = %f" % (data))
    sys.stdout.flush()
    value = map(data, 0, 1023, 0, 100)
    LED.ChangeDutyCycle(value)
    time.sleep( 0.01 )

ambient = 452.000000

KeyboardInterrupt: 

-----

110

110のサンプル

In [55]:
TILTPIN = 5

In [56]:
while True:
        if(GPIO.input(TILTPIN)):
            GPIO.output(LEDPIN, True)
        else:
            GPIO.output(LEDPIN, False)

UnboundLocalError: local variable 'led_state' referenced before assignment

toggle


UnboundLocalError: local variable 'led_state' referenced before assignment

UnboundLocalError: local variable 'led_state' referenced before assignment

toggle


...

UnboundLocalError: local variable 'led_state' referenced before assignment

UnboundLocalError: local variable 'led_state' referenced before assignment

toggle


UnboundLocalError: local variable 'led_state' referenced before assignment

KeyboardInterrupt: 

------------
116

116のサンプル

In [57]:
DISTANCEPIN = 0

In [59]:
while True:
        data = readadc(DISTANCEPIN)
        # 取得した値を電圧(mv)に変換
        volt = map(data, 0, 1023, 0, 5000)
        # 電圧から距離(cm)に変換
        distance = map(volt, 3200, 500, 5, 80)
        sys.stdout.write("\rdistance = %f" % (distance))
        sys.stdout.flush()
        time.sleep(0.05)

distance = 37.0000000

KeyboardInterrupt: 